!pip install requests pandas folium

# But et étapes de ce notebook

## But : Créer un pipeline permettant de relier un déchet ménager à un "trash score"

In [ ]:
Questions : 
--> TRR / TRV en %, kesako?
--> Broyeurs?


In [ ]:
Questions résolues : 
--> SINOE : https://www.sinoe.org

## Liens à explorer

### Europe 

### France 

### Mailles 

#### National

In [ ]:
https://www.sinoe.org/thematiques/consult/ss-theme/1

#### Regional

In [ ]:
https://www.sinoe.org/filtres/index/thematique#table-annuaire

#### Ville

#### Entreprise

#### Consommateur

# Code

In [1]:
import requests
import pandas as pd
import folium

In [2]:
# Create df from API call
def create_df(url):

    # Send a request to the API and store the response
    response = requests.get(url)
    
    if response.status_code == 200:
        # Convert JSON response to a Python dictionary
        data = response.json()

        # Extract records from the data and create a pandas DataFrame
        records = data["records"]
        df = pd.DataFrame([record["fields"] for record in records])
        return df
    
    else:
        return f"Error: {response.status_code}"
    

## Centres de recyclage 

In [3]:
df_centres_recyclage = create_df("https://opendata.bordeaux-metropole.fr/api/records/1.0/search/?dataset=dechetteries-en-temps-reel&q=&facet=statut&facet=insee")


In [11]:
df_centres_recyclage.nom.value_counts()

CR Blanquefort               1
CR Merignac                  1
CR Bassens EXT               1
CR Eysines                   1
CR Le Taillan                1
CR Bordeaux Bastide          1
CR Saint-Medard-en-Jalles    1
CR Surcouf                   1
CR Ambes                     1
CR Pessac                    1
Name: nom, dtype: int64

In [42]:
df_centres_recyclage[:1]

,nom,ident,geo_point_2d,horaires_osm,geo_shape,mdate,cdate,adresse,statut,geom_o,gid,acceptes,insee,refuses
0,CR Blanquefort,CR Blanquefort,"[44.9274953, -0.6228383]","Mo 08:30-12:30,13:15-18:00; Tu 13:15-18:00; We...","{'coordinates': [-0.6228383, 44.9274953], 'typ...",2023-04-19,2023-04-19,"Avenue du 11 Novembre, 33290 Blanquefort",OUVERT,0,1,TOUT_VENANT_INCINERABLE;TOUT_VENANT_NON_INCINE...,33056,AMIANTE;BOUTEILLES_DE_GAZ;DECHETS_DELEMENTS_DA...


## Dechets et exutoires 

In [13]:
df_dechets = create_df("https://opendata.bordeaux-metropole.fr/api/records/1.0/search/?dataset=met_dechets_exutoires_conseils&q=&facet=nom_dechet&facet=famille_dechet&facet=type&rows=10000")


In [14]:
df_dechets

,id_dechet,famille_dechet,nom_dechet,type,label
0,8,Produits chimiques,Acide,Exutoire,Déchetterie
1,14,Plastique,Agitateur en plastique,Exutoire,Poubelle noire
2,27,Textiles,Alèse,Exutoire;Conseil;Exutoire,Borne textile;Don / Entraide / Revente;Déchett...
3,30,Vie domestique,Allume feu,Exutoire,Poubelle noire
4,32,Papier carton,Almanach,Exutoire;Conseil,Poubelle verte;Don / Entraide / Revente
...,...,...,...,...,...
1443,1379,Papier carton,Tube en carton,Exutoire;Conseil,Poubelle verte;Réutilisation
1444,1396,Transport,Véhicule,Conseil;Conseil;Exutoire,Don / Entraide / Revente;Réparation;Ferrailleu...
1445,1399,Sport,Vélo d'intérieur,Conseil;Conseil;Exutoire;Conseil,Réparation;Location;Déchetterie;Don / Entraide...
1446,1405,Vie domestique,Verre de table,Exutoire;Conseil,Poubelle noire;Don / Entraide / Revente


## Relations dechets et exutoires

In [15]:
df_relation_dechets_exut = create_df("https://opendata.bordeaux-metropole.fr/api/records/1.0/search/?dataset=met_dechets&q=&rows=10000")


In [17]:
df_relation_dechets_exut

,id_dechet,nom_dechet
0,3,Abricot
1,8,Acide
2,14,Agitateur en plastique
3,17,Agrume
4,19,Aiguille de couture
...,...,...
1443,1420,Visière de protection
1444,1422,Vitre en verre
1445,1428,Vtc
1446,1429,VTT


## Wording Dechets 

In [18]:
df_wording = create_df("https://opendata.bordeaux-metropole.fr/api/records/1.0/search/?dataset=met_exutoires_conseils&q=&facet&rows=10000")

In [20]:
df_wording

,id_exutoire,type,description,label
0,1,Exutoire,Ce déchet est recyclable. Attention : les déch...,Poubelle verte
1,2,Exutoire,Ce déchet n'est pour le moment pas recyclé.,Poubelle noire
2,3,Exutoire,Vous pouvez jeter ce déchet dans la borne à ve...,Borne à verre
3,4,Exutoire,vous pouvez apporter ce déchet en déchèteries.,Déchetterie
4,5,Exutoire,Afin de récupérer les textiles usagés ou que v...,Borne textile
5,6,Exutoire,30 % des ordures ménagères peuvent être recycl...,Compostage
6,9,Exutoire,"Même s'il y a un reste de médicament, ne serai...",Cyclamed
7,12,Exutoire,Vous pouvez les apporter dans des bornes de co...,Electronique - Grandes ou moyennes surfaces
8,15,Exutoire,Ce déchet doit être déposé chez un professionn...,Déchetterie professionnelle
9,16,Exutoire,Ce déchet est collecté dans des points de coll...,Points de collecte provisoires


## Bornes recyclage verre

In [21]:
df_bornes_verre = create_df("https://opendata.bordeaux-metropole.fr/api/records/1.0/search/?dataset=tal_bornes_collecte_verre&q=&rows=10000")    


In [22]:
df_bornes_verre

,geo_point_2d,geometry
0,"[44.844412, -0.797011]","{'coordinates': [-0.797011, 44.844412], 'type'..."
1,"[44.768781, -0.595476]","{'coordinates': [-0.595476, 44.768781], 'type'..."
2,"[44.763133, -0.626047]","{'coordinates': [-0.626047, 44.763133], 'type'..."
3,"[44.771803, -0.626471]","{'coordinates': [-0.626471, 44.771803], 'type'..."
4,"[44.878105, -0.681235]","{'coordinates': [-0.681235, 44.878105], 'type'..."
...,...,...
962,"[44.871333, -0.50784]","{'coordinates': [-0.50784, 44.871333], 'type':..."
963,"[44.800456, -0.635113]","{'coordinates': [-0.635113, 44.800456], 'type'..."
964,"[44.87128, -0.509267]","{'coordinates': [-0.509267, 44.87128], 'type':..."
965,"[44.865702, -0.595171]","{'coordinates': [-0.595171, 44.865702], 'type'..."
